# 기본 옵션

In [1]:
import pandas as pd
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

pd.set_option('display.max.colwidth',60)
pd.options.display.max_columns = 100

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time
from bs4 import BeautifulSoup
import re

# 반려동물

## 1. Arirang은 고양이들내가 주인

https://www.youtube.com/channel/UCETBLykCDpfP6L0awBd7Vwg/videos

#### 영상  정보 수집 ( 채널명/ 썸네일/ 구독자 수/ 제목 / 업로드 날짜 / 조회수 / 좋아요 / 싫어요 / 영상길이 / 카테고리)
- 전체 영상 목록이 보이는 페이지에서 각 동영상의 url 추출하는 과정

In [3]:
# 원하는 유튜버 '동영상'에 들어가기 
driver = webdriver.Chrome('@needs_for_you/chromedriver.exe')
driver.maximize_window()
driver.get('https://www.youtube.com/channel/UCETBLykCDpfP6L0awBd7Vwg/videos')
body = driver.find_element_by_tag_name('body')
time.sleep(1.5)#브라우저 로딩시간기다려야함

# 유튜브는 스크롤해야 전체를 볼 수 있음 # 전체 영상 개수에 따라 조절해야함
endk = 110 # 5번 내림 # 500개면 한 100번정도 넉넉히 내리기 # 시험해보려고 1번만 내림
while endk:
    driver.find_element_by_tag_name('body').send_keys(Keys.END)
    time.sleep(0.3)
    endk -= 1    
start_page = driver.page_source 
soup = BeautifulSoup(start_page,'lxml') # 전체 영상 목록의 page-source 이니 soup으로

# 각 영상 페이지의 url 모음
video_list = soup.find_all('ytd-grid-video-renderer',{'class':'style-scope ytd-grid-renderer'})

arirangcat_url = []
base_url = 'http://www.youtube.com'
for i in range(len(video_list)):
    url = base_url+video_list[i].find('a',{'id':'thumbnail'})['href']
    arirangcat_url.append(url)

In [5]:
len(arirangcat_url)

263

- 추출한 모든 영상의 url을 입력하여, 각 페이지 안으로 들어가 정보를 수집
- 채널명/ 구독자 수 / 썸네일 / 제목 / 업로드 날짜 / 조회수 / 좋아요 / 싫어요 / 영상길이 / 카테고리

In [6]:
arirangcat = pd.DataFrame({'channel_name' :[],
                           'subscribers' : [],
                           'thumbnail' :[],
                           'video_name':[],
                           'video_length':[],
                           'upload_date':[],
                           'hits':[],
                           'likes_num':[],
                           'dislikes_num':[],
                           'category':[] })

for i in range(0,len(arirangcat_url)):
    
    try :# 오류생겨서 try-except 추가        
        channel_name = soup.find('title').text
        subscribers = soup.find('yt-formatted-string',{'id':'subscriber-count'}).text
        thumbnail = video_list[i].find('img', {'src' : True})['src'] 
        video_name = video_list[i].find('a',{'id':'video-title'}).text
        video_length = video_list[i].find('span', {'class':'style-scope ytd-thumbnail-overlay-time-status-renderer'}).text.split('\n')[1]

        # 각 동영상 페이지 들어가기 시작 
        start_url = arirangcat_url[i]
        driver.get(start_url) 
        body = driver.find_element_by_tag_name('body')

        time.sleep(1.5)

        # 댓글까지 쭉 보려면 스크롤 # 사실상 지금 댓글은 안보니 1번만 내리기
        endk = 1 # 5번 내림
        while endk:
            driver.find_element_by_tag_name('body').send_keys(Keys.END)
            time.sleep(0.3) # 내리는 시간 간격 
            endk -= 1    
        page = driver.page_source 
        html = BeautifulSoup(page,'lxml') # 각 페이지의 page_source 이니 html로


        # 각 동영상 페이지 들어가서 얻을 수 있는 정보
        pre_upload_date = html.find('div',{'id':'date'})
        upload_date = pre_upload_date.find('yt-formatted-string', {'class':'style-scope ytd-video-primary-info-renderer'}).text  
        pre_hits = html.find('yt-view-count-renderer', {'class':'style-scope ytd-video-primary-info-renderer'})
        hits = pre_hits.find('span',{'class':'view-count style-scope yt-view-count-renderer'}).text.split()[1]
        likes_num = html.find('yt-formatted-string',{'id':'text','class':'style-scope ytd-toggle-button-renderer style-text','aria-label':re.compile('좋아요')}).text+'개'
        dislikes_num = html.find('yt-formatted-string',{'id':'text','class':'style-scope ytd-toggle-button-renderer style-text','aria-label':re.compile('싫어요')}).text+'개'
        category = 'pets' # 바꾸면됨


        # 마지막에 모은 정보 합치기
        insert_data = pd.DataFrame({'channel_name' :[channel_name],
                                   'subscribers' : [subscribers],
                                   'thumbnail' :[thumbnail],
                                   'video_name':[video_name],
                                   'video_length':[video_length],
                                   'upload_date':[upload_date],
                                   'hits' :[hits],
                                   'likes_num':[likes_num],
                                   'dislikes_num':[dislikes_num],
                                   'category':[category]})
    
        arirangcat = arirangcat.append(insert_data)
    
    except KeyError as e: # error뜬거는 포함 안하고 저장됨 (error 뜨는거 찾을 때 마다 그냥 넣어버리기)
        print(k, i)
    except AttributeError as a: 
        print(a ,i)
    except IndexError as e:
        print(e,i)
    except TypeError as t:
        print(t, i)

'NoneType' object has no attribute 'find' 0
'NoneType' object has no attribute 'find' 1
'NoneType' object has no attribute 'find' 2
'NoneType' object has no attribute 'find' 3
'NoneType' object has no attribute 'find' 4
'NoneType' object has no attribute 'find' 5
'NoneType' object has no attribute 'find' 6
'NoneType' object has no attribute 'find' 7
'NoneType' object has no attribute 'find' 8
'NoneType' object has no attribute 'find' 9
'NoneType' object has no attribute 'find' 10
'NoneType' object has no attribute 'find' 11
'NoneType' object has no attribute 'find' 12
'NoneType' object has no attribute 'find' 13
'NoneType' object has no attribute 'find' 14
'NoneType' object has no attribute 'find' 15
'NoneType' object has no attribute 'find' 16
'NoneType' object has no attribute 'find' 17
'NoneType' object has no attribute 'find' 18
'NoneType' object has no attribute 'find' 19
'NoneType' object has no attribute 'find' 20
'NoneType' object has no attribute 'find' 21
'NoneType' object ha

In [11]:
print("shape : ",arirangcat.shape)
arirangcat.to_csv('arirangcat.csv', encoding='utf-8-sig')

shape :  (178, 10)


## 2. 갑수목장gabsupasture

https://www.youtube.com/channel/UC99OELa9yvqgkq9ffBvm9iQ/videos

In [14]:
# 원하는 유튜버 '동영상'에 들어가기 
driver = webdriver.Chrome('@needs_for_you/chromedriver.exe')
driver.maximize_window()
driver.get('https://www.youtube.com/channel/UC99OELa9yvqgkq9ffBvm9iQ/videos')
body = driver.find_element_by_tag_name('body')
time.sleep(1.5)#브라우저 로딩시간기다려야함

# 유튜브는 스크롤해야 전체를 볼 수 있음 # 전체 영상 개수에 따라 조절해야함
endk = 110 # 5번 내림 # 500개면 한 100번정도 넉넉히 내리기 # 시험해보려고 1번만 내림
while endk:
    driver.find_element_by_tag_name('body').send_keys(Keys.END)
    time.sleep(0.3)
    endk -= 1    
start_page = driver.page_source 
soup = BeautifulSoup(start_page,'lxml') # 전체 영상 목록의 page-source 이니 soup으로

# 각 영상 페이지의 url 모음
video_list = soup.find_all('ytd-grid-video-renderer',{'class':'style-scope ytd-grid-renderer'})

gabsupasture_url = []
base_url = 'http://www.youtube.com'
for i in range(len(video_list)):
    url = base_url+video_list[i].find('a',{'id':'thumbnail'})['href']
    gabsupasture_url.append(url)

In [15]:
len(gabsupasture_url)

693

In [16]:
gabsupasture = pd.DataFrame({'channel_name' :[],
                           'subscribers' : [],
                           'thumbnail' :[],
                           'video_name':[],
                           'video_length':[],
                           'upload_date':[],
                           'hits':[],
                           'likes_num':[],
                           'dislikes_num':[],
                           'category':[] })

for i in range(0,len(gabsupasture_url)):
    
    try :# 오류생겨서 try-except 추가        
        channel_name = soup.find('title').text
        subscribers = soup.find('yt-formatted-string',{'id':'subscriber-count'}).text
        thumbnail = video_list[i].find('img', {'src' : True})['src'] 
        video_name = video_list[i].find('a',{'id':'video-title'}).text
        video_length = video_list[i].find('span', {'class':'style-scope ytd-thumbnail-overlay-time-status-renderer'}).text.split('\n')[1]

        # 각 동영상 페이지 들어가기 시작 
        start_url = gabsupasture_url[i]
        driver.get(start_url) 
        body = driver.find_element_by_tag_name('body')

        time.sleep(1.5)

        # 댓글까지 쭉 보려면 스크롤 # 사실상 지금 댓글은 안보니 1번만 내리기
        endk = 1 # 5번 내림
        while endk:
            driver.find_element_by_tag_name('body').send_keys(Keys.END)
            time.sleep(0.3) # 내리는 시간 간격 
            endk -= 1    
        page = driver.page_source 
        html = BeautifulSoup(page,'lxml') # 각 페이지의 page_source 이니 html로


        # 각 동영상 페이지 들어가서 얻을 수 있는 정보
        pre_upload_date = html.find('div',{'id':'date'})
        upload_date = pre_upload_date.find('yt-formatted-string', {'class':'style-scope ytd-video-primary-info-renderer'}).text  
        pre_hits = html.find('yt-view-count-renderer', {'class':'style-scope ytd-video-primary-info-renderer'})
        hits = pre_hits.find('span',{'class':'view-count style-scope yt-view-count-renderer'}).text.split()[1]
        likes_num = html.find('yt-formatted-string',{'id':'text','class':'style-scope ytd-toggle-button-renderer style-text','aria-label':re.compile('좋아요')}).text+'개'
        dislikes_num = html.find('yt-formatted-string',{'id':'text','class':'style-scope ytd-toggle-button-renderer style-text','aria-label':re.compile('싫어요')}).text+'개'
        category = 'pets' # 바꾸면됨


        # 마지막에 모은 정보 합치기
        insert_data = pd.DataFrame({'channel_name' :[channel_name],
                                   'subscribers' : [subscribers],
                                   'thumbnail' :[thumbnail],
                                   'video_name':[video_name],
                                   'video_length':[video_length],
                                   'upload_date':[upload_date],
                                   'hits' :[hits],
                                   'likes_num':[likes_num],
                                   'dislikes_num':[dislikes_num],
                                   'category':[category]})
    
        gabsupasture = gabsupasture.append(insert_data)
    
    except KeyError as e: # error뜬거는 포함 안하고 저장됨 (error 뜨는거 찾을 때 마다 그냥 넣어버리기)
        print(k, i)
    except AttributeError as a: 
        print(a ,i)
    except IndexError as e:
        print(e,i)
    except TypeError as t:
        print(t, i)

'NoneType' object has no attribute 'find' 0
'NoneType' object has no attribute 'find' 1
'NoneType' object has no attribute 'find' 2
'NoneType' object has no attribute 'find' 3
'NoneType' object has no attribute 'find' 4
'NoneType' object has no attribute 'find' 5
'NoneType' object has no attribute 'find' 6
'NoneType' object has no attribute 'find' 7
'NoneType' object has no attribute 'find' 8
'NoneType' object has no attribute 'find' 9
'NoneType' object has no attribute 'find' 10
'NoneType' object has no attribute 'find' 11
'NoneType' object has no attribute 'find' 12
'NoneType' object has no attribute 'find' 13
'NoneType' object has no attribute 'find' 14
'NoneType' object has no attribute 'find' 15
'NoneType' object has no attribute 'find' 16
'NoneType' object has no attribute 'find' 17
'NoneType' object has no attribute 'find' 18
'NoneType' object is not subscriptable 42
'NoneType' object is not subscriptable 43
'NoneType' object is not subscriptable 44
'NoneType' object is not subs

In [17]:
print("shape : ",gabsupasture.shape)
gabsupasture.to_csv('gabsupasture.csv', encoding='utf-8-sig')

shape :  (644, 10)


## 3. 시바견 곰이탱이여우

https://www.youtube.com/channel/UCX2laRqGQhqoChYmlaUgOiw/videos

In [25]:
# 원하는 유튜버 '동영상'에 들어가기 
driver = webdriver.Chrome('@needs_for_you/chromedriver.exe')
driver.maximize_window()
driver.get('https://www.youtube.com/channel/UCX2laRqGQhqoChYmlaUgOiw/videos')
body = driver.find_element_by_tag_name('body')
time.sleep(1.5)#브라우저 로딩시간기다려야함

# 유튜브는 스크롤해야 전체를 볼 수 있음 # 전체 영상 개수에 따라 조절해야함
endk = 110 # 5번 내림 # 500개면 한 100번정도 넉넉히 내리기 # 시험해보려고 1번만 내림
while endk:
    driver.find_element_by_tag_name('body').send_keys(Keys.END)
    time.sleep(0.3)
    endk -= 1    
start_page = driver.page_source 
soup = BeautifulSoup(start_page,'lxml') # 전체 영상 목록의 page-source 이니 soup으로

# 각 영상 페이지의 url 모음
video_list = soup.find_all('ytd-grid-video-renderer',{'class':'style-scope ytd-grid-renderer'})

gomttangfox_url = []
base_url = 'http://www.youtube.com'
for i in range(len(video_list)):
    url = base_url+video_list[i].find('a',{'id':'thumbnail'})['href']
    gomttangfox_url.append(url)

In [26]:
len(gomttangfox_url)

842

In [27]:
gomttangfox = pd.DataFrame({'channel_name' :[],
                           'subscribers' : [],
                           'thumbnail' :[],
                           'video_name':[],
                           'video_length':[],
                           'upload_date':[],
                           'hits':[],
                           'likes_num':[],
                           'dislikes_num':[],
                           'category':[] })

for i in range(0,len(gomttangfox_url)):
    
    try :# 오류생겨서 try-except 추가        
        channel_name = soup.find('title').text
        subscribers = soup.find('yt-formatted-string',{'id':'subscriber-count'}).text
        thumbnail = video_list[i].find('img', {'src' : True})['src'] 
        video_name = video_list[i].find('a',{'id':'video-title'}).text
        video_length = video_list[i].find('span', {'class':'style-scope ytd-thumbnail-overlay-time-status-renderer'}).text.split('\n')[1]

        # 각 동영상 페이지 들어가기 시작 
        start_url = gomttangfox_url[i]
        driver.get(start_url) 
        body = driver.find_element_by_tag_name('body')

        time.sleep(1.5)

        # 댓글까지 쭉 보려면 스크롤 # 사실상 지금 댓글은 안보니 1번만 내리기
        endk = 1 # 5번 내림
        while endk:
            driver.find_element_by_tag_name('body').send_keys(Keys.END)
            time.sleep(0.3) # 내리는 시간 간격 
            endk -= 1    
        page = driver.page_source 
        html = BeautifulSoup(page,'lxml') # 각 페이지의 page_source 이니 html로


        # 각 동영상 페이지 들어가서 얻을 수 있는 정보
        pre_upload_date = html.find('div',{'id':'date'})
        upload_date = pre_upload_date.find('yt-formatted-string', {'class':'style-scope ytd-video-primary-info-renderer'}).text  
        pre_hits = html.find('yt-view-count-renderer', {'class':'style-scope ytd-video-primary-info-renderer'})
        hits = pre_hits.find('span',{'class':'view-count style-scope yt-view-count-renderer'}).text.split()[1]
        likes_num = html.find('yt-formatted-string',{'id':'text','class':'style-scope ytd-toggle-button-renderer style-text','aria-label':re.compile('좋아요')}).text+'개'
        dislikes_num = html.find('yt-formatted-string',{'id':'text','class':'style-scope ytd-toggle-button-renderer style-text','aria-label':re.compile('싫어요')}).text+'개'
        category = 'pets' # 바꾸면됨


        # 마지막에 모은 정보 합치기
        insert_data = pd.DataFrame({'channel_name' :[channel_name],
                                   'subscribers' : [subscribers],
                                   'thumbnail' :[thumbnail],
                                   'video_name':[video_name],
                                   'video_length':[video_length],
                                   'upload_date':[upload_date],
                                   'hits' :[hits],
                                   'likes_num':[likes_num],
                                   'dislikes_num':[dislikes_num],
                                   'category':[category]})
    
        gomttangfox = gomttangfox.append(insert_data)
    
    except KeyError as e: # error뜬거는 포함 안하고 저장됨 (error 뜨는거 찾을 때 마다 그냥 넣어버리기)
        print(k, i)
    except AttributeError as a: 
        print(a ,i)
    except IndexError as e:
        print(e,i)
    except TypeError as t:
        print(t, i)

'NoneType' object is not subscriptable 221
'NoneType' object is not subscriptable 222
'NoneType' object is not subscriptable 223
'NoneType' object is not subscriptable 224
'NoneType' object is not subscriptable 311
'NoneType' object is not subscriptable 312
'NoneType' object is not subscriptable 313
'NoneType' object is not subscriptable 314
'NoneType' object is not subscriptable 341
'NoneType' object is not subscriptable 342
'NoneType' object is not subscriptable 343
'NoneType' object is not subscriptable 344
'NoneType' object is not subscriptable 461
'NoneType' object is not subscriptable 462
'NoneType' object is not subscriptable 463
'NoneType' object is not subscriptable 464
'NoneType' object is not subscriptable 730
'NoneType' object is not subscriptable 731
'NoneType' object is not subscriptable 732
'NoneType' object is not subscriptable 733
'NoneType' object is not subscriptable 734
'NoneType' object is not subscriptable 760
'NoneType' object is not subscriptable 761
'NoneType' 

In [28]:
print("shape : ",gomttangfox.shape)
gomttangfox.to_csv('gomttangfox.csv', encoding='utf-8-sig')

shape :  (816, 10)


## 4. haha ha

https://www.youtube.com/channel/UCOp66Vup07X0YziXaaxqs2A/videos

In [3]:
# 원하는 유튜버 '동영상'에 들어가기 
driver = webdriver.Chrome('@needs_for_you/chromedriver.exe')
driver.maximize_window()
driver.get('https://www.youtube.com/channel/UCOp66Vup07X0YziXaaxqs2A/videos')
body = driver.find_element_by_tag_name('body')
time.sleep(1.5)#브라우저 로딩시간기다려야함

# 유튜브는 스크롤해야 전체를 볼 수 있음 # 전체 영상 개수에 따라 조절해야함
endk = 110 # 5번 내림 # 500개면 한 100번정도 넉넉히 내리기 # 시험해보려고 1번만 내림
while endk:
    driver.find_element_by_tag_name('body').send_keys(Keys.END)
    time.sleep(0.3)
    endk -= 1    
start_page = driver.page_source 
soup = BeautifulSoup(start_page,'lxml') # 전체 영상 목록의 page-source 이니 soup으로

# 각 영상 페이지의 url 모음
video_list = soup.find_all('ytd-grid-video-renderer',{'class':'style-scope ytd-grid-renderer'})

hahaha_url = []
base_url = 'http://www.youtube.com'
for i in range(len(video_list)):
    url = base_url+video_list[i].find('a',{'id':'thumbnail'})['href']
    hahaha_url.append(url)

In [4]:
len(hahaha_url)

274

In [5]:
hahaha = pd.DataFrame({'channel_name' :[],
                           'subscribers' : [],
                           'thumbnail' :[],
                           'video_name':[],
                           'video_length':[],
                           'upload_date':[],
                           'hits':[],
                           'likes_num':[],
                           'dislikes_num':[],
                           'category':[] })

for i in range(0,len(hahaha_url)):
    
    try :# 오류생겨서 try-except 추가        
        channel_name = soup.find('title').text
        subscribers = soup.find('yt-formatted-string',{'id':'subscriber-count'}).text
        thumbnail = video_list[i].find('img', {'src' : True})['src'] 
        video_name = video_list[i].find('a',{'id':'video-title'}).text
        video_length = video_list[i].find('span', {'class':'style-scope ytd-thumbnail-overlay-time-status-renderer'}).text.split('\n')[1]

        # 각 동영상 페이지 들어가기 시작 
        start_url = hahaha_url[i]
        driver.get(start_url) 
        body = driver.find_element_by_tag_name('body')

        time.sleep(1.5)

        # 댓글까지 쭉 보려면 스크롤 # 사실상 지금 댓글은 안보니 1번만 내리기
        endk = 1 # 5번 내림
        while endk:
            driver.find_element_by_tag_name('body').send_keys(Keys.END)
            time.sleep(0.3) # 내리는 시간 간격 
            endk -= 1    
        page = driver.page_source 
        html = BeautifulSoup(page,'lxml') # 각 페이지의 page_source 이니 html로


        # 각 동영상 페이지 들어가서 얻을 수 있는 정보
        pre_upload_date = html.find('div',{'id':'date'})
        upload_date = pre_upload_date.find('yt-formatted-string', {'class':'style-scope ytd-video-primary-info-renderer'}).text  
        pre_hits = html.find('yt-view-count-renderer', {'class':'style-scope ytd-video-primary-info-renderer'})
        hits = pre_hits.find('span',{'class':'view-count style-scope yt-view-count-renderer'}).text.split()[1]
        likes_num = html.find('yt-formatted-string',{'id':'text','class':'style-scope ytd-toggle-button-renderer style-text','aria-label':re.compile('좋아요')}).text+'개'
        dislikes_num = html.find('yt-formatted-string',{'id':'text','class':'style-scope ytd-toggle-button-renderer style-text','aria-label':re.compile('싫어요')}).text+'개'
        category = 'pets' # 바꾸면됨


        # 마지막에 모은 정보 합치기
        insert_data = pd.DataFrame({'channel_name' :[channel_name],
                                   'subscribers' : [subscribers],
                                   'thumbnail' :[thumbnail],
                                   'video_name':[video_name],
                                   'video_length':[video_length],
                                   'upload_date':[upload_date],
                                   'hits' :[hits],
                                   'likes_num':[likes_num],
                                   'dislikes_num':[dislikes_num],
                                   'category':[category]})
    
        hahaha = hahaha.append(insert_data)
    
    except KeyError as e: # error뜬거는 포함 안하고 저장됨 (error 뜨는거 찾을 때 마다 그냥 넣어버리기)
        print(k, i)
    except AttributeError as a: 
        print(a ,i)
    except IndexError as e:
        print(e,i)
    except TypeError as t:
        print(t, i)

'NoneType' object is not subscriptable 42
'NoneType' object is not subscriptable 43
'NoneType' object is not subscriptable 44
'NoneType' object is not subscriptable 72
'NoneType' object is not subscriptable 73
'NoneType' object is not subscriptable 74
'NoneType' object is not subscriptable 102
'NoneType' object is not subscriptable 103
'NoneType' object is not subscriptable 104
'NoneType' object has no attribute 'find' 120
'NoneType' object is not subscriptable 132
'NoneType' object is not subscriptable 133
'NoneType' object is not subscriptable 134
'NoneType' object is not subscriptable 222
'NoneType' object is not subscriptable 223
'NoneType' object is not subscriptable 224


In [6]:
print("shape : ",hahaha.shape)
hahaha.to_csv('hahaha.csv', encoding='utf-8-sig')

shape :  (258, 10)


## 5. mochamilk

https://www.youtube.com/channel/UCnjyiWHGEyww-p8QYSftx2A/videos

In [4]:
# 원하는 유튜버 '동영상'에 들어가기 
driver = webdriver.Chrome('@needs_for_you/chromedriver.exe')
driver.maximize_window()
driver.get('https://www.youtube.com/channel/UCnjyiWHGEyww-p8QYSftx2A/videos')
body = driver.find_element_by_tag_name('body')
time.sleep(1.5)#브라우저 로딩시간기다려야함

# 유튜브는 스크롤해야 전체를 볼 수 있음 # 전체 영상 개수에 따라 조절해야함
endk = 30 # 5번 내림 # 500개면 한 100번정도 넉넉히 내리기 # 시험해보려고 1번만 내림
while endk:
    driver.find_element_by_tag_name('body').send_keys(Keys.END)
    time.sleep(0.3)
    endk -= 1    
start_page = driver.page_source 
soup = BeautifulSoup(start_page,'lxml') # 전체 영상 목록의 page-source 이니 soup으로

# 각 영상 페이지의 url 모음
video_list = soup.find_all('ytd-grid-video-renderer',{'class':'style-scope ytd-grid-renderer'})

mochamilk_url = []
base_url = 'http://www.youtube.com'
for i in range(len(video_list)):
    url = base_url+video_list[i].find('a',{'id':'thumbnail'})['href']
    mochamilk_url.append(url)

In [5]:
len(mochamilk_url)

116

In [6]:
mochamilk = pd.DataFrame({'channel_name' :[],
                           'subscribers' : [],
                           'thumbnail' :[],
                           'video_name':[],
                           'video_length':[],
                           'upload_date':[],
                           'hits':[],
                           'likes_num':[],
                           'dislikes_num':[],
                           'category':[] })

for i in range(0,len(mochamilk_url)):
    
    try :# 오류생겨서 try-except 추가        
        channel_name = soup.find('title').text
        subscribers = soup.find('yt-formatted-string',{'id':'subscriber-count'}).text
        thumbnail = video_list[i].find('img', {'src' : True})['src'] 
        video_name = video_list[i].find('a',{'id':'video-title'}).text
        video_length = video_list[i].find('span', {'class':'style-scope ytd-thumbnail-overlay-time-status-renderer'}).text.split('\n')[1]

        # 각 동영상 페이지 들어가기 시작 
        start_url = mochamilk_url[i]
        driver.get(start_url) 
        body = driver.find_element_by_tag_name('body')

        time.sleep(1.5)

        # 댓글까지 쭉 보려면 스크롤 # 사실상 지금 댓글은 안보니 1번만 내리기
        endk = 1 # 5번 내림
        while endk:
            driver.find_element_by_tag_name('body').send_keys(Keys.END)
            time.sleep(0.3) # 내리는 시간 간격 
            endk -= 1    
        page = driver.page_source 
        html = BeautifulSoup(page,'lxml') # 각 페이지의 page_source 이니 html로


        # 각 동영상 페이지 들어가서 얻을 수 있는 정보
        pre_upload_date = html.find('div',{'id':'date'})
        upload_date = pre_upload_date.find('yt-formatted-string', {'class':'style-scope ytd-video-primary-info-renderer'}).text  
        pre_hits = html.find('yt-view-count-renderer', {'class':'style-scope ytd-video-primary-info-renderer'})
        hits = pre_hits.find('span',{'class':'view-count style-scope yt-view-count-renderer'}).text.split()[1]
        likes_num = html.find('yt-formatted-string',{'id':'text','class':'style-scope ytd-toggle-button-renderer style-text','aria-label':re.compile('좋아요')}).text+'개'
        dislikes_num = html.find('yt-formatted-string',{'id':'text','class':'style-scope ytd-toggle-button-renderer style-text','aria-label':re.compile('싫어요')}).text+'개'
        category = 'pets' # 바꾸면됨


        # 마지막에 모은 정보 합치기
        insert_data = pd.DataFrame({'channel_name' :[channel_name],
                                   'subscribers' : [subscribers],
                                   'thumbnail' :[thumbnail],
                                   'video_name':[video_name],
                                   'video_length':[video_length],
                                   'upload_date':[upload_date],
                                   'hits' :[hits],
                                   'likes_num':[likes_num],
                                   'dislikes_num':[dislikes_num],
                                   'category':[category]})
    
        mochamilk = mochamilk.append(insert_data)
    
    except KeyError as e: # error뜬거는 포함 안하고 저장됨 (error 뜨는거 찾을 때 마다 그냥 넣어버리기)
        print(k, i)
    except AttributeError as a: 
        print(a ,i)
    except IndexError as e:
        print(e,i)
    except TypeError as t:
        print(t, i)

'NoneType' object has no attribute 'find' 0
'NoneType' object has no attribute 'find' 1
'NoneType' object has no attribute 'find' 2
'NoneType' object has no attribute 'find' 3
'NoneType' object has no attribute 'find' 4
'NoneType' object has no attribute 'find' 5
'NoneType' object has no attribute 'find' 6
'NoneType' object has no attribute 'find' 7
'NoneType' object has no attribute 'find' 8
'NoneType' object has no attribute 'find' 9
'NoneType' object has no attribute 'find' 10
'NoneType' object has no attribute 'find' 11


In [7]:
print("shape : ",mochamilk.shape)
mochamilk.to_csv('mochamilk.csv', encoding='utf-8-sig')

shape :  (104, 10)


## 6. 소녀의행성

https://www.youtube.com/channel/UC-IYpdOqjkER7B6cnChwlhA/videos

In [8]:
# 원하는 유튜버 '동영상'에 들어가기 
driver = webdriver.Chrome('@needs_for_you/chromedriver.exe')
driver.maximize_window()
driver.get('https://www.youtube.com/channel/UC-IYpdOqjkER7B6cnChwlhA/videos')
body = driver.find_element_by_tag_name('body')
time.sleep(1.5)#브라우저 로딩시간기다려야함

# 유튜브는 스크롤해야 전체를 볼 수 있음 # 전체 영상 개수에 따라 조절해야함
endk = 65 # 5번 내림 # 500개면 한 100번정도 넉넉히 내리기 # 시험해보려고 1번만 내림
while endk:
    driver.find_element_by_tag_name('body').send_keys(Keys.END)
    time.sleep(0.3)
    endk -= 1    
start_page = driver.page_source 
soup = BeautifulSoup(start_page,'lxml') # 전체 영상 목록의 page-source 이니 soup으로

# 각 영상 페이지의 url 모음
video_list = soup.find_all('ytd-grid-video-renderer',{'class':'style-scope ytd-grid-renderer'})

sohaengseong_url = []
base_url = 'http://www.youtube.com'
for i in range(len(video_list)):
    url = base_url+video_list[i].find('a',{'id':'thumbnail'})['href']
    sohaengseong_url.append(url)

In [9]:
len(sohaengseong_url)

241

In [10]:
sohaengseong = pd.DataFrame({'channel_name' :[],
                           'subscribers' : [],
                           'thumbnail' :[],
                           'video_name':[],
                           'video_length':[],
                           'upload_date':[],
                           'hits':[],
                           'likes_num':[],
                           'dislikes_num':[],
                           'category':[] })

for i in range(0,len(sohaengseong_url)):
    
    try :# 오류생겨서 try-except 추가        
        channel_name = soup.find('title').text
        subscribers = soup.find('yt-formatted-string',{'id':'subscriber-count'}).text
        thumbnail = video_list[i].find('img', {'src' : True})['src'] 
        video_name = video_list[i].find('a',{'id':'video-title'}).text
        video_length = video_list[i].find('span', {'class':'style-scope ytd-thumbnail-overlay-time-status-renderer'}).text.split('\n')[1]

        # 각 동영상 페이지 들어가기 시작 
        start_url = sohaengseong_url[i]
        driver.get(start_url) 
        body = driver.find_element_by_tag_name('body')

        time.sleep(1.5)

        # 댓글까지 쭉 보려면 스크롤 # 사실상 지금 댓글은 안보니 1번만 내리기
        endk = 1 # 5번 내림
        while endk:
            driver.find_element_by_tag_name('body').send_keys(Keys.END)
            time.sleep(0.3) # 내리는 시간 간격 
            endk -= 1    
        page = driver.page_source 
        html = BeautifulSoup(page,'lxml') # 각 페이지의 page_source 이니 html로


        # 각 동영상 페이지 들어가서 얻을 수 있는 정보
        pre_upload_date = html.find('div',{'id':'date'})
        upload_date = pre_upload_date.find('yt-formatted-string', {'class':'style-scope ytd-video-primary-info-renderer'}).text  
        pre_hits = html.find('yt-view-count-renderer', {'class':'style-scope ytd-video-primary-info-renderer'})
        hits = pre_hits.find('span',{'class':'view-count style-scope yt-view-count-renderer'}).text.split()[1]
        likes_num = html.find('yt-formatted-string',{'id':'text','class':'style-scope ytd-toggle-button-renderer style-text','aria-label':re.compile('좋아요')}).text+'개'
        dislikes_num = html.find('yt-formatted-string',{'id':'text','class':'style-scope ytd-toggle-button-renderer style-text','aria-label':re.compile('싫어요')}).text+'개'
        category = 'pets' # 바꾸면됨


        # 마지막에 모은 정보 합치기
        insert_data = pd.DataFrame({'channel_name' :[channel_name],
                                   'subscribers' : [subscribers],
                                   'thumbnail' :[thumbnail],
                                   'video_name':[video_name],
                                   'video_length':[video_length],
                                   'upload_date':[upload_date],
                                   'hits' :[hits],
                                   'likes_num':[likes_num],
                                   'dislikes_num':[dislikes_num],
                                   'category':[category]})
    
        sohaengseong = sohaengseong.append(insert_data)
    
    except KeyError as e: # error뜬거는 포함 안하고 저장됨 (error 뜨는거 찾을 때 마다 그냥 넣어버리기)
        print(k, i)
    except AttributeError as a: 
        print(a ,i)
    except IndexError as e:
        print(e,i)
    except TypeError as t:
        print(t, i)

'NoneType' object is not subscriptable 132
'NoneType' object is not subscriptable 133
'NoneType' object is not subscriptable 134
'NoneType' object is not subscriptable 162
'NoneType' object is not subscriptable 163
'NoneType' object is not subscriptable 164
'NoneType' object is not subscriptable 192
'NoneType' object is not subscriptable 193
'NoneType' object is not subscriptable 194


In [11]:
print("shape : ",sohaengseong.shape)
sohaengseong.to_csv('sohaengseong.csv', encoding='utf-8-sig')

shape :  (232, 10)


## 7. 22똥괭이네

https://www.youtube.com/channel/UCYaBl0gtXV_3sHW4bQjq0hA/videos

In [15]:
# 원하는 유튜버 '동영상'에 들어가기 
driver = webdriver.Chrome('@needs_for_you/chromedriver.exe')
driver.maximize_window()
driver.get('https://www.youtube.com/channel/UCYaBl0gtXV_3sHW4bQjq0hA/videos')
body = driver.find_element_by_tag_name('body')
time.sleep(1.5)#브라우저 로딩시간기다려야함

# 유튜브는 스크롤해야 전체를 볼 수 있음 # 전체 영상 개수에 따라 조절해야함
endk = 110 # 5번 내림 # 500개면 한 100번정도 넉넉히 내리기 # 시험해보려고 1번만 내림
while endk:
    driver.find_element_by_tag_name('body').send_keys(Keys.END)
    time.sleep(0.3)
    endk -= 1    
start_page = driver.page_source 
soup = BeautifulSoup(start_page,'lxml') # 전체 영상 목록의 page-source 이니 soup으로

# 각 영상 페이지의 url 모음
video_list = soup.find_all('ytd-grid-video-renderer',{'class':'style-scope ytd-grid-renderer'})

ddongcat_url = []
base_url = 'http://www.youtube.com'
for i in range(len(video_list)):
    url = base_url+video_list[i].find('a',{'id':'thumbnail'})['href']
    ddongcat_url.append(url)

In [16]:
len(ddongcat_url)

692

In [17]:
ddongcat = pd.DataFrame({'channel_name' :[],
                           'subscribers' : [],
                           'thumbnail' :[],
                           'video_name':[],
                           'video_length':[],
                           'upload_date':[],
                           'hits':[],
                           'likes_num':[],
                           'dislikes_num':[],
                           'category':[] })

for i in range(0,len(ddongcat_url)):
    
    try :# 오류생겨서 try-except 추가        
        channel_name = soup.find('title').text
        subscribers = soup.find('yt-formatted-string',{'id':'subscriber-count'}).text
        thumbnail = video_list[i].find('img', {'src' : True})['src'] 
        video_name = video_list[i].find('a',{'id':'video-title'}).text
        video_length = video_list[i].find('span', {'class':'style-scope ytd-thumbnail-overlay-time-status-renderer'}).text.split('\n')[1]

        # 각 동영상 페이지 들어가기 시작 
        start_url = ddongcat_url[i]
        driver.get(start_url) 
        body = driver.find_element_by_tag_name('body')

        time.sleep(1.5)

        # 댓글까지 쭉 보려면 스크롤 # 사실상 지금 댓글은 안보니 1번만 내리기
        endk = 1 # 5번 내림
        while endk:
            driver.find_element_by_tag_name('body').send_keys(Keys.END)
            time.sleep(0.3) # 내리는 시간 간격 
            endk -= 1    
        page = driver.page_source 
        html = BeautifulSoup(page,'lxml') # 각 페이지의 page_source 이니 html로


        # 각 동영상 페이지 들어가서 얻을 수 있는 정보
        pre_upload_date = html.find('div',{'id':'date'})
        upload_date = pre_upload_date.find('yt-formatted-string', {'class':'style-scope ytd-video-primary-info-renderer'}).text  
        pre_hits = html.find('yt-view-count-renderer', {'class':'style-scope ytd-video-primary-info-renderer'})
        hits = pre_hits.find('span',{'class':'view-count style-scope yt-view-count-renderer'}).text.split()[1]
        likes_num = html.find('yt-formatted-string',{'id':'text','class':'style-scope ytd-toggle-button-renderer style-text','aria-label':re.compile('좋아요')}).text+'개'
        dislikes_num = html.find('yt-formatted-string',{'id':'text','class':'style-scope ytd-toggle-button-renderer style-text','aria-label':re.compile('싫어요')}).text+'개'
        category = 'pets' # 바꾸면됨


        # 마지막에 모은 정보 합치기
        insert_data = pd.DataFrame({'channel_name' :[channel_name],
                                   'subscribers' : [subscribers],
                                   'thumbnail' :[thumbnail],
                                   'video_name':[video_name],
                                   'video_length':[video_length],
                                   'upload_date':[upload_date],
                                   'hits' :[hits],
                                   'likes_num':[likes_num],
                                   'dislikes_num':[dislikes_num],
                                   'category':[category]})
    
        ddongcat = ddongcat.append(insert_data)
    
    except KeyError as e: # error뜬거는 포함 안하고 저장됨 (error 뜨는거 찾을 때 마다 그냥 넣어버리기)
        print(k, i)
    except AttributeError as a: 
        print(a ,i)
    except IndexError as e:
        print(e,i)
    except TypeError as t:
        print(t, i)

'NoneType' object is not subscriptable 102
'NoneType' object is not subscriptable 103
'NoneType' object is not subscriptable 104
'NoneType' object is not subscriptable 192
'NoneType' object is not subscriptable 193
'NoneType' object is not subscriptable 194
'NoneType' object is not subscriptable 312
'NoneType' object is not subscriptable 313
'NoneType' object is not subscriptable 314
'NoneType' object is not subscriptable 372
'NoneType' object is not subscriptable 373
'NoneType' object is not subscriptable 374
'NoneType' object is not subscriptable 462
'NoneType' object is not subscriptable 463
'NoneType' object is not subscriptable 464
'NoneType' object is not subscriptable 492
'NoneType' object is not subscriptable 493
'NoneType' object is not subscriptable 494
'NoneType' object is not subscriptable 551
'NoneType' object is not subscriptable 552
'NoneType' object is not subscriptable 553
'NoneType' object is not subscriptable 554
'NoneType' object is not subscriptable 611
'NoneType' 

In [18]:
print("shape : ",ddongcat.shape)
ddongcat.to_csv('ddongcat.csv', encoding='utf-8-sig')

shape :  (666, 10)


## 8. 냥이아빠

https://www.youtube.com/channel/UC5AAf4_zZxk-mCl46TogZQQ/videos

In [19]:
# 원하는 유튜버 '동영상'에 들어가기 
driver = webdriver.Chrome('@needs_for_you/chromedriver.exe')
driver.maximize_window()
driver.get('https://www.youtube.com/channel/UC5AAf4_zZxk-mCl46TogZQQ/videos')
body = driver.find_element_by_tag_name('body')
time.sleep(1.5)#브라우저 로딩시간기다려야함

# 유튜브는 스크롤해야 전체를 볼 수 있음 # 전체 영상 개수에 따라 조절해야함
endk = 110 # 5번 내림 # 500개면 한 100번정도 넉넉히 내리기 # 시험해보려고 1번만 내림
while endk:
    driver.find_element_by_tag_name('body').send_keys(Keys.END)
    time.sleep(0.3)
    endk -= 1    
start_page = driver.page_source 
soup = BeautifulSoup(start_page,'lxml') # 전체 영상 목록의 page-source 이니 soup으로

# 각 영상 페이지의 url 모음
video_list = soup.find_all('ytd-grid-video-renderer',{'class':'style-scope ytd-grid-renderer'})

catdaddy_url = []
base_url = 'http://www.youtube.com'
for i in range(len(video_list)):
    url = base_url+video_list[i].find('a',{'id':'thumbnail'})['href']
    catdaddy_url.append(url)

In [20]:
len(catdaddy_url)

231

In [21]:
catdaddy = pd.DataFrame({'channel_name' :[],
                           'subscribers' : [],
                           'thumbnail' :[],
                           'video_name':[],
                           'video_length':[],
                           'upload_date':[],
                           'hits':[],
                           'likes_num':[],
                           'dislikes_num':[],
                           'category':[] })

for i in range(0,len(catdaddy_url)):
    
    try :# 오류생겨서 try-except 추가        
        channel_name = soup.find('title').text
        subscribers = soup.find('yt-formatted-string',{'id':'subscriber-count'}).text
        thumbnail = video_list[i].find('img', {'src' : True})['src'] 
        video_name = video_list[i].find('a',{'id':'video-title'}).text
        video_length = video_list[i].find('span', {'class':'style-scope ytd-thumbnail-overlay-time-status-renderer'}).text.split('\n')[1]

        # 각 동영상 페이지 들어가기 시작 
        start_url = catdaddy_url[i]
        driver.get(start_url) 
        body = driver.find_element_by_tag_name('body')

        time.sleep(1.5)

        # 댓글까지 쭉 보려면 스크롤 # 사실상 지금 댓글은 안보니 1번만 내리기
        endk = 1 # 5번 내림
        while endk:
            driver.find_element_by_tag_name('body').send_keys(Keys.END)
            time.sleep(0.3) # 내리는 시간 간격 
            endk -= 1    
        page = driver.page_source 
        html = BeautifulSoup(page,'lxml') # 각 페이지의 page_source 이니 html로


        # 각 동영상 페이지 들어가서 얻을 수 있는 정보
        pre_upload_date = html.find('div',{'id':'date'})
        upload_date = pre_upload_date.find('yt-formatted-string', {'class':'style-scope ytd-video-primary-info-renderer'}).text  
        pre_hits = html.find('yt-view-count-renderer', {'class':'style-scope ytd-video-primary-info-renderer'})
        hits = pre_hits.find('span',{'class':'view-count style-scope yt-view-count-renderer'}).text.split()[1]
        likes_num = html.find('yt-formatted-string',{'id':'text','class':'style-scope ytd-toggle-button-renderer style-text','aria-label':re.compile('좋아요')}).text+'개'
        dislikes_num = html.find('yt-formatted-string',{'id':'text','class':'style-scope ytd-toggle-button-renderer style-text','aria-label':re.compile('싫어요')}).text+'개'
        category = 'pets' # 바꾸면됨


        # 마지막에 모은 정보 합치기
        insert_data = pd.DataFrame({'channel_name' :[channel_name],
                                   'subscribers' : [subscribers],
                                   'thumbnail' :[thumbnail],
                                   'video_name':[video_name],
                                   'video_length':[video_length],
                                   'upload_date':[upload_date],
                                   'hits' :[hits],
                                   'likes_num':[likes_num],
                                   'dislikes_num':[dislikes_num],
                                   'category':[category]})
    
        catdaddy = catdaddy.append(insert_data)
    
    except KeyError as e: # error뜬거는 포함 안하고 저장됨 (error 뜨는거 찾을 때 마다 그냥 넣어버리기)
        print(k, i)
    except AttributeError as a: 
        print(a ,i)
    except IndexError as e:
        print(e,i)
    except TypeError as t:
        print(t, i)

'NoneType' object has no attribute 'find' 0
'NoneType' object has no attribute 'find' 1
'NoneType' object has no attribute 'find' 2
'NoneType' object has no attribute 'find' 3
'NoneType' object has no attribute 'find' 4
'NoneType' object has no attribute 'find' 5
'NoneType' object has no attribute 'find' 6
'NoneType' object has no attribute 'find' 7
'NoneType' object has no attribute 'find' 8
'NoneType' object has no attribute 'find' 9
'NoneType' object has no attribute 'find' 10
'NoneType' object has no attribute 'find' 11
'NoneType' object has no attribute 'find' 12
'NoneType' object has no attribute 'find' 13
'NoneType' object has no attribute 'find' 14
'NoneType' object has no attribute 'find' 15
'NoneType' object has no attribute 'find' 16
'NoneType' object has no attribute 'find' 17
'NoneType' object has no attribute 'find' 18
'NoneType' object has no attribute 'find' 19
'NoneType' object has no attribute 'find' 20
'NoneType' object has no attribute 'find' 21
'NoneType' object ha

In [22]:
print("shape : ",catdaddy.shape)
catdaddy.to_csv('catdaddy.csv', encoding='utf-8-sig')

shape :  (176, 10)


## 9. 밀키와 복이MilkyBoki

https://www.youtube.com/channel/UCP1CQH2PVJv4-glNTuVJl3g/videos

In [19]:
# 원하는 유튜버 '동영상'에 들어가기 
driver = webdriver.Chrome('@needs_for_you/chromedriver.exe')
driver.maximize_window()
driver.get('https://www.youtube.com/channel/UCP1CQH2PVJv4-glNTuVJl3g/videos')
body = driver.find_element_by_tag_name('body')
time.sleep(1.5)#브라우저 로딩시간기다려야함

# 유튜브는 스크롤해야 전체를 볼 수 있음 # 전체 영상 개수에 따라 조절해야함
endk = 110 # 5번 내림 # 500개면 한 100번정도 넉넉히 내리기 # 시험해보려고 1번만 내림
while endk:
    driver.find_element_by_tag_name('body').send_keys(Keys.END)
    time.sleep(0.3)
    endk -= 1    
start_page = driver.page_source 
soup = BeautifulSoup(start_page,'lxml') # 전체 영상 목록의 page-source 이니 soup으로

# 각 영상 페이지의 url 모음
video_list = soup.find_all('ytd-grid-video-renderer',{'class':'style-scope ytd-grid-renderer'})

milkyboki_url = []
base_url = 'http://www.youtube.com'
for i in range(len(video_list)):
    url = base_url+video_list[i].find('a',{'id':'thumbnail'})['href']
    milkyboki_url.append(url)

In [20]:
len(milkyboki_url)

146

In [21]:
milkyboki = pd.DataFrame({'channel_name' :[],
                           'subscribers' : [],
                           'thumbnail' :[],
                           'video_name':[],
                           'video_length':[],
                           'upload_date':[],
                           'hits':[],
                           'likes_num':[],
                           'dislikes_num':[],
                           'category':[] })

for i in range(0,len(milkyboki_url)):
    
    try :# 오류생겨서 try-except 추가        
        channel_name = soup.find('title').text
        subscribers = soup.find('yt-formatted-string',{'id':'subscriber-count'}).text
        thumbnail = video_list[i].find('img', {'src' : True})['src'] 
        video_name = video_list[i].find('a',{'id':'video-title'}).text
        video_length = video_list[i].find('span', {'class':'style-scope ytd-thumbnail-overlay-time-status-renderer'}).text.split('\n')[1]

        # 각 동영상 페이지 들어가기 시작 
        start_url = milkyboki_url[i]
        driver.get(start_url) 
        body = driver.find_element_by_tag_name('body')

        time.sleep(1.5)

        # 댓글까지 쭉 보려면 스크롤 # 사실상 지금 댓글은 안보니 1번만 내리기
        endk = 1 # 5번 내림
        while endk:
            driver.find_element_by_tag_name('body').send_keys(Keys.END)
            time.sleep(0.3) # 내리는 시간 간격 
            endk -= 1    
        page = driver.page_source 
        html = BeautifulSoup(page,'lxml') # 각 페이지의 page_source 이니 html로


        # 각 동영상 페이지 들어가서 얻을 수 있는 정보
        pre_upload_date = html.find('div',{'id':'date'})
        upload_date = pre_upload_date.find('yt-formatted-string', {'class':'style-scope ytd-video-primary-info-renderer'}).text  
        pre_hits = html.find('yt-view-count-renderer', {'class':'style-scope ytd-video-primary-info-renderer'})
        hits = pre_hits.find('span',{'class':'view-count style-scope yt-view-count-renderer'}).text.split()[1]
        likes_num = html.find('yt-formatted-string',{'id':'text','class':'style-scope ytd-toggle-button-renderer style-text','aria-label':re.compile('좋아요')}).text+'개'
        dislikes_num = html.find('yt-formatted-string',{'id':'text','class':'style-scope ytd-toggle-button-renderer style-text','aria-label':re.compile('싫어요')}).text+'개'
        category = 'pets' # 바꾸면됨


        # 마지막에 모은 정보 합치기
        insert_data = pd.DataFrame({'channel_name' :[channel_name],
                                   'subscribers' : [subscribers],
                                   'thumbnail' :[thumbnail],
                                   'video_name':[video_name],
                                   'video_length':[video_length],
                                   'upload_date':[upload_date],
                                   'hits' :[hits],
                                   'likes_num':[likes_num],
                                   'dislikes_num':[dislikes_num],
                                   'category':[category]})
    
        milkyboki = milkyboki.append(insert_data)
    
    except KeyError as e: # error뜬거는 포함 안하고 저장됨 (error 뜨는거 찾을 때 마다 그냥 넣어버리기)
        print(k, i)
    except AttributeError as a: 
        print(a ,i)
    except IndexError as e:
        print(e,i)
    except TypeError as t:
        print(t, i)

In [22]:
print("shape : ",milkyboki.shape)
milkyboki.to_csv('milkyboki.csv', encoding='utf-8-sig')

shape :  (146, 10)


## 10. 뭉땡뭉땡

https://www.youtube.com/channel/UCzay0uHEiaSsKmuGkCg-mSg/videos

In [23]:
# 원하는 유튜버 '동영상'에 들어가기 
driver = webdriver.Chrome('@needs_for_you/chromedriver.exe')
driver.maximize_window()
driver.get('https://www.youtube.com/channel/UCzay0uHEiaSsKmuGkCg-mSg/videos')
body = driver.find_element_by_tag_name('body')
time.sleep(1.5)#브라우저 로딩시간기다려야함

# 유튜브는 스크롤해야 전체를 볼 수 있음 # 전체 영상 개수에 따라 조절해야함
endk = 110 # 5번 내림 # 500개면 한 100번정도 넉넉히 내리기 # 시험해보려고 1번만 내림
while endk:
    driver.find_element_by_tag_name('body').send_keys(Keys.END)
    time.sleep(0.3)
    endk -= 1    
start_page = driver.page_source 
soup = BeautifulSoup(start_page,'lxml') # 전체 영상 목록의 page-source 이니 soup으로

# 각 영상 페이지의 url 모음
video_list = soup.find_all('ytd-grid-video-renderer',{'class':'style-scope ytd-grid-renderer'})

mungttaeng_url = []
base_url = 'http://www.youtube.com'
for i in range(len(video_list)):
    url = base_url+video_list[i].find('a',{'id':'thumbnail'})['href']
    mungttaeng_url.append(url)

In [24]:
len(mungttaeng_url)

306

In [25]:
mungttaeng = pd.DataFrame({'channel_name' :[],
                           'subscribers' : [],
                           'thumbnail' :[],
                           'video_name':[],
                           'video_length':[],
                           'upload_date':[],
                           'hits':[],
                           'likes_num':[],
                           'dislikes_num':[],
                           'category':[] })

for i in range(0,len(mungttaeng_url)):
    
    try :# 오류생겨서 try-except 추가        
        channel_name = soup.find('title').text
        subscribers = soup.find('yt-formatted-string',{'id':'subscriber-count'}).text
        thumbnail = video_list[i].find('img', {'src' : True})['src'] 
        video_name = video_list[i].find('a',{'id':'video-title'}).text
        video_length = video_list[i].find('span', {'class':'style-scope ytd-thumbnail-overlay-time-status-renderer'}).text.split('\n')[1]

        # 각 동영상 페이지 들어가기 시작 
        start_url = mungttaeng_url[i]
        driver.get(start_url) 
        body = driver.find_element_by_tag_name('body')

        time.sleep(1.5)

        # 댓글까지 쭉 보려면 스크롤 # 사실상 지금 댓글은 안보니 1번만 내리기
        endk = 1 # 5번 내림
        while endk:
            driver.find_element_by_tag_name('body').send_keys(Keys.END)
            time.sleep(0.3) # 내리는 시간 간격 
            endk -= 1    
        page = driver.page_source 
        html = BeautifulSoup(page,'lxml') # 각 페이지의 page_source 이니 html로


        # 각 동영상 페이지 들어가서 얻을 수 있는 정보
        pre_upload_date = html.find('div',{'id':'date'})
        upload_date = pre_upload_date.find('yt-formatted-string', {'class':'style-scope ytd-video-primary-info-renderer'}).text  
        pre_hits = html.find('yt-view-count-renderer', {'class':'style-scope ytd-video-primary-info-renderer'})
        hits = pre_hits.find('span',{'class':'view-count style-scope yt-view-count-renderer'}).text.split()[1]
        likes_num = html.find('yt-formatted-string',{'id':'text','class':'style-scope ytd-toggle-button-renderer style-text','aria-label':re.compile('좋아요')}).text+'개'
        dislikes_num = html.find('yt-formatted-string',{'id':'text','class':'style-scope ytd-toggle-button-renderer style-text','aria-label':re.compile('싫어요')}).text+'개'
        category = 'pets' # 바꾸면됨


        # 마지막에 모은 정보 합치기
        insert_data = pd.DataFrame({'channel_name' :[channel_name],
                                   'subscribers' : [subscribers],
                                   'thumbnail' :[thumbnail],
                                   'video_name':[video_name],
                                   'video_length':[video_length],
                                   'upload_date':[upload_date],
                                   'hits' :[hits],
                                   'likes_num':[likes_num],
                                   'dislikes_num':[dislikes_num],
                                   'category':[category]})
    
        mungttaeng = mungttaeng.append(insert_data)
    
    except KeyError as e: # error뜬거는 포함 안하고 저장됨 (error 뜨는거 찾을 때 마다 그냥 넣어버리기)
        print(k, i)
    except AttributeError as a: 
        print(a ,i)
    except IndexError as e:
        print(e,i)
    except TypeError as t:
        print(t, i)

'NoneType' object is not subscriptable 18
'NoneType' object is not subscriptable 19
'NoneType' object is not subscriptable 76
'NoneType' object is not subscriptable 77
'NoneType' object is not subscriptable 78
'NoneType' object is not subscriptable 79
'NoneType' object is not subscriptable 136
'NoneType' object is not subscriptable 137
'NoneType' object is not subscriptable 138
'NoneType' object is not subscriptable 139
'NoneType' object is not subscriptable 192
'NoneType' object is not subscriptable 193
'NoneType' object is not subscriptable 194
'NoneType' object is not subscriptable 195
'NoneType' object is not subscriptable 222
'NoneType' object is not subscriptable 223
'NoneType' object is not subscriptable 252
'NoneType' object is not subscriptable 253
'NoneType' object is not subscriptable 254
'NoneType' object is not subscriptable 255
'NoneType' object is not subscriptable 256
'NoneType' object is not subscriptable 257
'NoneType' object is not subscriptable 258
'NoneType' object

In [26]:
print("shape : ",mungttaeng.shape)
mungttaeng.to_csv('mungttaeng.csv', encoding='utf-8-sig')

shape :  (278, 10)


## 11. 김메주와 고양이들

https://www.youtube.com/channel/UCCb6W2FU1L7j9mw14YK-9yg/videos

In [36]:
# 원하는 유튜버 '동영상'에 들어가기 
driver = webdriver.Chrome('@needs_for_you/chromedriver.exe')
driver.maximize_window()
driver.get('https://www.youtube.com/channel/UCCb6W2FU1L7j9mw14YK-9yg/videos')
body = driver.find_element_by_tag_name('body')
time.sleep(1.5)#브라우저 로딩시간기다려야함

# 유튜브는 스크롤해야 전체를 볼 수 있음 # 전체 영상 개수에 따라 조절해야함
endk = 110 # 5번 내림 # 500개면 한 100번정도 넉넉히 내리기 # 시험해보려고 1번만 내림
while endk:
    driver.find_element_by_tag_name('body').send_keys(Keys.END)
    time.sleep(0.3)
    endk -= 1    
start_page = driver.page_source 
soup = BeautifulSoup(start_page,'lxml') # 전체 영상 목록의 page-source 이니 soup으로

# 각 영상 페이지의 url 모음
video_list = soup.find_all('ytd-grid-video-renderer',{'class':'style-scope ytd-grid-renderer'})

maejucat_url = []
base_url = 'http://www.youtube.com'
for i in range(len(video_list)):
    url = base_url+video_list[i].find('a',{'id':'thumbnail'})['href']
    maejucat_url.append(url)

In [37]:
len(maejucat_url)

763

In [38]:
maejucat = pd.DataFrame({'channel_name' :[],
                           'subscribers' : [],
                           'thumbnail' :[],
                           'video_name':[],
                           'video_length':[],
                           'upload_date':[],
                           'hits':[],
                           'likes_num':[],
                           'dislikes_num':[],
                           'category':[] })

for i in range(0,len(maejucat_url)):
    
    try :# 오류생겨서 try-except 추가        
        channel_name = soup.find('title').text
        subscribers = soup.find('yt-formatted-string',{'id':'subscriber-count'}).text
        thumbnail = video_list[i].find('img', {'src' : True})['src'] 
        video_name = video_list[i].find('a',{'id':'video-title'}).text
        video_length = video_list[i].find('span', {'class':'style-scope ytd-thumbnail-overlay-time-status-renderer'}).text.split('\n')[1]

        # 각 동영상 페이지 들어가기 시작 
        start_url = maejucat_url[i]
        driver.get(start_url) 
        body = driver.find_element_by_tag_name('body')

        time.sleep(1.5)

        # 댓글까지 쭉 보려면 스크롤 # 사실상 지금 댓글은 안보니 1번만 내리기
        endk = 1 # 5번 내림
        while endk:
            driver.find_element_by_tag_name('body').send_keys(Keys.END)
            time.sleep(0.3) # 내리는 시간 간격 
            endk -= 1    
        page = driver.page_source 
        html = BeautifulSoup(page,'lxml') # 각 페이지의 page_source 이니 html로


        # 각 동영상 페이지 들어가서 얻을 수 있는 정보
        pre_upload_date = html.find('div',{'id':'date'})
        upload_date = pre_upload_date.find('yt-formatted-string', {'class':'style-scope ytd-video-primary-info-renderer'}).text  
        pre_hits = html.find('yt-view-count-renderer', {'class':'style-scope ytd-video-primary-info-renderer'})
        hits = pre_hits.find('span',{'class':'view-count style-scope yt-view-count-renderer'}).text.split()[1]
        likes_num = html.find('yt-formatted-string',{'id':'text','class':'style-scope ytd-toggle-button-renderer style-text','aria-label':re.compile('좋아요')}).text+'개'
        dislikes_num = html.find('yt-formatted-string',{'id':'text','class':'style-scope ytd-toggle-button-renderer style-text','aria-label':re.compile('싫어요')}).text+'개'
        category = 'pets' # 바꾸면됨


        # 마지막에 모은 정보 합치기
        insert_data = pd.DataFrame({'channel_name' :[channel_name],
                                   'subscribers' : [subscribers],
                                   'thumbnail' :[thumbnail],
                                   'video_name':[video_name],
                                   'video_length':[video_length],
                                   'upload_date':[upload_date],
                                   'hits' :[hits],
                                   'likes_num':[likes_num],
                                   'dislikes_num':[dislikes_num],
                                   'category':[category]})
    
        maejucat = maejucat.append(insert_data)
    
    except KeyError as e: # error뜬거는 포함 안하고 저장됨 (error 뜨는거 찾을 때 마다 그냥 넣어버리기)
        print(k, i)
    except AttributeError as a: 
        print(a ,i)
    except IndexError as e:
        print(e,i)
    except TypeError as t:
        print(t, i)

'NoneType' object is not subscriptable 70
'NoneType' object is not subscriptable 71
'NoneType' object is not subscriptable 72
'NoneType' object is not subscriptable 73
'NoneType' object is not subscriptable 74
'NoneType' object is not subscriptable 220
'NoneType' object is not subscriptable 221
'NoneType' object is not subscriptable 222
'NoneType' object is not subscriptable 223
'NoneType' object is not subscriptable 224
'NoneType' object is not subscriptable 340
'NoneType' object is not subscriptable 341
'NoneType' object is not subscriptable 342
'NoneType' object is not subscriptable 343
'NoneType' object is not subscriptable 344
'NoneType' object is not subscriptable 550
'NoneType' object is not subscriptable 551
'NoneType' object is not subscriptable 552
'NoneType' object is not subscriptable 553
'NoneType' object is not subscriptable 554
'NoneType' object is not subscriptable 730
'NoneType' object is not subscriptable 731
'NoneType' object is not subscriptable 732
'NoneType' objec

In [39]:
print("shape : ",maejucat.shape)
maejucat.to_csv('maejucat.csv', encoding='utf-8-sig')

shape :  (737, 10)


## 12. 모찌모찌 mochi

https://www.youtube.com/channel/UCuXmuXtdLEDjZ0G9m03iYaQ/videos

In [30]:
# 원하는 유튜버 '동영상'에 들어가기 
driver = webdriver.Chrome('@needs_for_you/chromedriver.exe')
driver.maximize_window()
driver.get('https://www.youtube.com/channel/UCuXmuXtdLEDjZ0G9m03iYaQ/videos')
body = driver.find_element_by_tag_name('body')
time.sleep(1.5)#브라우저 로딩시간기다려야함

# 유튜브는 스크롤해야 전체를 볼 수 있음 # 전체 영상 개수에 따라 조절해야함
endk = 110 # 5번 내림 # 500개면 한 100번정도 넉넉히 내리기 # 시험해보려고 1번만 내림
while endk:
    driver.find_element_by_tag_name('body').send_keys(Keys.END)
    time.sleep(0.3)
    endk -= 1    
start_page = driver.page_source 
soup = BeautifulSoup(start_page,'lxml') # 전체 영상 목록의 page-source 이니 soup으로

# 각 영상 페이지의 url 모음
video_list = soup.find_all('ytd-grid-video-renderer',{'class':'style-scope ytd-grid-renderer'})

mochi_url = []
base_url = 'http://www.youtube.com'
for i in range(len(video_list)):
    url = base_url+video_list[i].find('a',{'id':'thumbnail'})['href']
    mochi_url.append(url)

# 동영상 개수 확인    
len(mochi_url)

273

In [31]:
mochi = pd.DataFrame({'channel_name' :[],
                           'subscribers' : [],
                           'thumbnail' :[],
                           'video_name':[],
                           'video_length':[],
                           'upload_date':[],
                           'hits':[],
                           'likes_num':[],
                           'dislikes_num':[],
                           'category':[] })

for i in range(0,len(mochi_url)):
    
    try :# 오류생겨서 try-except 추가        
        channel_name = soup.find('title').text
        subscribers = soup.find('yt-formatted-string',{'id':'subscriber-count'}).text
        thumbnail = video_list[i].find('img', {'src' : True})['src'] 
        video_name = video_list[i].find('a',{'id':'video-title'}).text
        video_length = video_list[i].find('span', {'class':'style-scope ytd-thumbnail-overlay-time-status-renderer'}).text.split('\n')[1]

        # 각 동영상 페이지 들어가기 시작 
        start_url = mochi_url[i]
        driver.get(start_url) 
        body = driver.find_element_by_tag_name('body')

        time.sleep(1.5)

        # 댓글까지 쭉 보려면 스크롤 # 사실상 지금 댓글은 안보니 1번만 내리기
        endk = 1 # 5번 내림
        while endk:
            driver.find_element_by_tag_name('body').send_keys(Keys.END)
            time.sleep(0.3) # 내리는 시간 간격 
            endk -= 1    
        page = driver.page_source 
        html = BeautifulSoup(page,'lxml') # 각 페이지의 page_source 이니 html로


        # 각 동영상 페이지 들어가서 얻을 수 있는 정보
        pre_upload_date = html.find('div',{'id':'date'})
        upload_date = pre_upload_date.find('yt-formatted-string', {'class':'style-scope ytd-video-primary-info-renderer'}).text  
        pre_hits = html.find('yt-view-count-renderer', {'class':'style-scope ytd-video-primary-info-renderer'})
        hits = pre_hits.find('span',{'class':'view-count style-scope yt-view-count-renderer'}).text.split()[1]
        likes_num = html.find('yt-formatted-string',{'id':'text','class':'style-scope ytd-toggle-button-renderer style-text','aria-label':re.compile('좋아요')}).text+'개'
        dislikes_num = html.find('yt-formatted-string',{'id':'text','class':'style-scope ytd-toggle-button-renderer style-text','aria-label':re.compile('싫어요')}).text+'개'
        category = 'pets' # 바꾸면됨


        # 마지막에 모은 정보 합치기
        insert_data = pd.DataFrame({'channel_name' :[channel_name],
                                   'subscribers' : [subscribers],
                                   'thumbnail' :[thumbnail],
                                   'video_name':[video_name],
                                   'video_length':[video_length],
                                   'upload_date':[upload_date],
                                   'hits' :[hits],
                                   'likes_num':[likes_num],
                                   'dislikes_num':[dislikes_num],
                                   'category':[category]})
    
        mochi = mochi.append(insert_data)
    
    except KeyError as e: # error뜬거는 포함 안하고 저장됨 (error 뜨는거 찾을 때 마다 그냥 넣어버리기)
        print(k, i)
    except AttributeError as a: 
        print(a ,i)
    except IndexError as e:
        print(e,i)
    except TypeError as t:
        print(t, i)

In [32]:
print("shape : ",mochi.shape)
mochi.to_csv('mochi.csv', encoding='utf-8-sig')

shape :  (273, 10)


## 13. 속삭이는몽자

https://www.youtube.com/user/CHUCHU7325/videos

In [33]:
# 원하는 유튜버 '동영상'에 들어가기 
driver = webdriver.Chrome('@needs_for_you/chromedriver.exe')
driver.maximize_window()
driver.get('https://www.youtube.com/user/CHUCHU7325/videos')
body = driver.find_element_by_tag_name('body')
time.sleep(1.5)#브라우저 로딩시간기다려야함

# 유튜브는 스크롤해야 전체를 볼 수 있음 # 전체 영상 개수에 따라 조절해야함
endk = 110 # 5번 내림 # 500개면 한 100번정도 넉넉히 내리기 # 시험해보려고 1번만 내림
while endk:
    driver.find_element_by_tag_name('body').send_keys(Keys.END)
    time.sleep(0.3)
    endk -= 1    
start_page = driver.page_source 
soup = BeautifulSoup(start_page,'lxml') # 전체 영상 목록의 page-source 이니 soup으로

# 각 영상 페이지의 url 모음
video_list = soup.find_all('ytd-grid-video-renderer',{'class':'style-scope ytd-grid-renderer'})

mongja_url = []
base_url = 'http://www.youtube.com'
for i in range(len(video_list)):
    url = base_url+video_list[i].find('a',{'id':'thumbnail'})['href']
    mongja_url.append(url)

# 동영상 개수 확인    
len(mongja_url)

375

In [34]:
mongja = pd.DataFrame({'channel_name' :[],
                           'subscribers' : [],
                           'thumbnail' :[],
                           'video_name':[],
                           'video_length':[],
                           'upload_date':[],
                           'hits':[],
                           'likes_num':[],
                           'dislikes_num':[],
                           'category':[] })

for i in range(0,len(mongja_url)):
    
    try :# 오류생겨서 try-except 추가        
        channel_name = soup.find('title').text
        subscribers = soup.find('yt-formatted-string',{'id':'subscriber-count'}).text
        thumbnail = video_list[i].find('img', {'src' : True})['src'] 
        video_name = video_list[i].find('a',{'id':'video-title'}).text
        video_length = video_list[i].find('span', {'class':'style-scope ytd-thumbnail-overlay-time-status-renderer'}).text.split('\n')[1]

        # 각 동영상 페이지 들어가기 시작 
        start_url = mongja_url[i]
        driver.get(start_url) 
        body = driver.find_element_by_tag_name('body')

        time.sleep(1.5)

        # 댓글까지 쭉 보려면 스크롤 # 사실상 지금 댓글은 안보니 1번만 내리기
        endk = 1 # 5번 내림
        while endk:
            driver.find_element_by_tag_name('body').send_keys(Keys.END)
            time.sleep(0.3) # 내리는 시간 간격 
            endk -= 1    
        page = driver.page_source 
        html = BeautifulSoup(page,'lxml') # 각 페이지의 page_source 이니 html로


        # 각 동영상 페이지 들어가서 얻을 수 있는 정보
        pre_upload_date = html.find('div',{'id':'date'})
        upload_date = pre_upload_date.find('yt-formatted-string', {'class':'style-scope ytd-video-primary-info-renderer'}).text  
        pre_hits = html.find('yt-view-count-renderer', {'class':'style-scope ytd-video-primary-info-renderer'})
        hits = pre_hits.find('span',{'class':'view-count style-scope yt-view-count-renderer'}).text.split()[1]
        likes_num = html.find('yt-formatted-string',{'id':'text','class':'style-scope ytd-toggle-button-renderer style-text','aria-label':re.compile('좋아요')}).text+'개'
        dislikes_num = html.find('yt-formatted-string',{'id':'text','class':'style-scope ytd-toggle-button-renderer style-text','aria-label':re.compile('싫어요')}).text+'개'
        category = 'pets' # 바꾸면됨


        # 마지막에 모은 정보 합치기
        insert_data = pd.DataFrame({'channel_name' :[channel_name],
                                   'subscribers' : [subscribers],
                                   'thumbnail' :[thumbnail],
                                   'video_name':[video_name],
                                   'video_length':[video_length],
                                   'upload_date':[upload_date],
                                   'hits' :[hits],
                                   'likes_num':[likes_num],
                                   'dislikes_num':[dislikes_num],
                                   'category':[category]})
    
        mongja = mongja.append(insert_data)
    
    except KeyError as e: # error뜬거는 포함 안하고 저장됨 (error 뜨는거 찾을 때 마다 그냥 넣어버리기)
        print(k, i)
    except AttributeError as a: 
        print(a ,i)
    except IndexError as e:
        print(e,i)
    except TypeError as t:
        print(t, i)

'NoneType' object is not subscriptable 72
'NoneType' object is not subscriptable 73
'NoneType' object is not subscriptable 74
'NoneType' object is not subscriptable 132
'NoneType' object is not subscriptable 133
'NoneType' object is not subscriptable 134
'NoneType' object is not subscriptable 162
'NoneType' object is not subscriptable 163
'NoneType' object is not subscriptable 164
'NoneType' object is not subscriptable 192
'NoneType' object is not subscriptable 193
'NoneType' object is not subscriptable 194


In [35]:
print("shape : ",mongja.shape)
mongja.to_csv('mongja.csv', encoding='utf-8-sig')

shape :  (363, 10)
